In [ ]:
# Imports
import numpy as np
import os
import csv
import json

In [ ]:
def addPedestrian(in_file, out_file, ped_id, x, y, target_ids, speed):
    ped = {
      "source" : None,
      "targetIds" : target_ids,
      "position" : {
        "x" : x,
        "y" : y
      },
      "velocity" : {
        "x" : 0.0,
        "y" : 0.0
      },
      "nextTargetListIndex" : 0,
      "freeFlowSpeed" : speed,
      "attributes" : {
        "id" : ped_id,
        "radius" : 0.2,
        "densityDependentSpeed" : False,
        "speedDistributionMean" : 1.34,
        "speedDistributionStandardDeviation" : 0.26,
        "minimumSpeed" : 0.5,
        "maximumSpeed" : 2.2,
        "acceleration" : 2.0,
        "footStepsToStore" : 4,
        "searchRadius" : 1.0,
        "angleCalculationType" : "USE_CENTER",
        "targetOrientationAngleThreshold" : 45.0
      },
      "idAsTarget" : -1,
      "isChild" : False,
      "isLikelyInjured" : False,
      "mostImportantEvent" : None,
      "salientBehavior" : "TARGET_ORIENTED",
      "groupIds" : [ ],
      "trajectory" : {
        "footSteps" : [ ]
      },
      "groupSizes" : [ ],
      "modelPedestrianMap" : { },
      "type" : "PEDESTRIAN"
    }
    with open('./scenarios/' + in_file + '.scenario', 'r') as infile:
        data = json.load(infile)
        data['name'] = out_file
        data['scenario']['topography']['dynamicElements'].append(ped)
    with open('./scenarios/' + out_file + '.scenario', 'w') as outfile:
        json.dump(data, outfile, indent=2)

In [ ]:
# Creating scenarios
def create_scenario(scenario_name, p_xy, p_v):
    with open('./scenarios/bottleneck_gnm.scenario', 'r') as infile:
        data = json.load(infile)
    with open('./scenarios/' + scenario_name + '.scenario', 'w') as new_sec:
        json.dump(data, new_sec, indent=2)
    peds = p_xy.reshape((-1,2))
    p_id = 1
    for p in peds:
        p_x = p[0]
        p_y = p[1]
        addPedestrian(scenario_name, scenario_name, p_id, p_x, p_y, [2], p_v[p_id-1])
        p_id += 1

In [ ]:
# Running scenarios
# !!! vadere-console.jar must be placed in the same folder as this notebook!!!
def run_scenario(scenario_name):
    java_command = 'java -jar vadere-console.jar scenario-run'
    command = java_command + ' --scenario-file "scenarios/' + scenario_name + '.scenario" --output-dir="output"'
    os.system(command)

In [ ]:
# Loading OSM
def load_postvis(sec_name):
    filename = './output/' + sec_name + '/postvis.trajectories'
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ')
        next(reader)
        data = []
        for row in reader:
            pedestrian = [int(row[0]), int(row[1]), float(row[2]), float(row[3]), int(row[4]), float(row[5])]
            # After step 29 the first pedestrian "disappears" inside the target
            if (int(row[0]) > 29): continue
            a = np.array(pedestrian)
            data.append(a)
        return np.row_stack(data)

In [ ]:
# Not required for the tasks, but used for testing
n = 3

In [ ]:
n = 15

In [ ]:
n = 20

In [ ]:
n = 25

In [ ]:
n = 30

In [ ]:
# Meta Information
osm_sec_name = 'bottleneck_osm_' + str(n)
matrix = load_postvis(osm_sec_name)
NT = int(max(matrix[:,0]))
m = 10
d = 2
nd = n*d

true_error = 1e-4

random_seed = 1
np.random.seed(random_seed)




# Initializing xt
xt = np.zeros((NT, (nd*m)))
# Pedestrian velocities
p_v = np.zeros((NT,n))
for r in matrix:
    for i in range(m):
        step = int(r[0]) - 1
        p_id = int(r[1]) - 1
        p_x = r[2] + np.random.normal(0,1)
        p_y = r[3] + np.random.normal(0,1)
        p_v[step,p_id] = r[5]
        xt[step,(p_id*d+i*nd):((p_id+1)*d+i*nd)] = np.array([p_x, p_y])

In [ ]:
def normal_draw(cov):
    """ draw an n-dimensional point from a Gaussian distribution with given covariance. """
    return np.random.multivariate_normal(0*cov[:,0],cov,n)

def observation(x):
    return x

def f_model(p_xy, ts):
    sec_name = 'f_hat'
    create_scenario(sec_name, p_xy, p_v[ts])
    run_scenario(sec_name)
    matrix = load_postvis(sec_name)
    result = np.zeros((nd,))
    for r in matrix:
        step = int(r[0]) - 1
        if (step == 0): continue
        p_id = int(r[1]) - 1
        p_x = r[2]
        p_y = r[3]
        result[(p_id*d):((p_id+1)*d)] = np.array([p_x, p_y])
    return result    

def enks(z_data, error_covariance_M, error_covariance_Q, observation, fhat_model):
    t = z_data.shape[0]
    ML = (error_covariance_M)
    QL = (error_covariance_Q)
    # initialize the initial guess for the model state with random numbers 
    xk = np.random.rand(z_data.shape[0],z_data.shape[1])+2
    
    for k in range(1,t):
        zk_hat = np.zeros((z_data.shape[1],))
        zk_sum = np.zeros(nd)
        for i in range(m):
            mkm1 = normal_draw(ML)
            xk[k,(i*nd):((i+1)*nd)] = fhat_model(xk[k-1,(i*nd):((i+1)*nd)].reshape(1,-1),k-1) + mkm1.reshape(1,-1)
            qk = normal_draw(QL)
            zk_hat[(i*nd):((i+1)*nd)] = observation(xk[k,(i*nd):((i+1)*nd)].reshape(1,-1)) + qk.reshape(1, -1)
            zk_sum += zk_hat[(i*nd):((i+1)*nd)]
            
        zk_bar = (1/m)*zk_sum
        
        zk_outer_sum = np.zeros((nd, nd))
        for i in range(m):
            zkdiff = zk_hat[(i*nd):((i+1)*nd)] - zk_bar
            outer = np.outer(zkdiff, zkdiff)
            zk_outer_sum += outer
        
        Zk = (1/m)*zk_outer_sum
        
        for j in range(1,k+1):
            # xjbar
            xj_hat_sum = np.zeros(nd)
            for i in range(m):
                xj_hat_sum += xk[j,(i*nd):((i+1)*nd)]
            xj_bar = (1/m)*xj_hat_sum
            
            # sigmaj
            outer_sum = np.zeros((nd,nd))
            for i in range(m):
                xj_diff = xk[j,(i*nd):((i+1)*nd)] - xj_bar
                zk_diff = zk_hat[(i*nd):((i+1)*nd)] - zk_bar
                outer_sum += np.outer(xj_diff, zk_diff)
            sigmaj = (1/(m-1))*outer_sum
            
            # reassigning x_hat
            Zk_inv = np.linalg.pinv(Zk,rcond=1e-10)
            mat_prod = sigmaj.dot(Zk_inv)
            for i in range(m):
                zk_diff = z_data[k,(i*nd):((i+1)*nd)] - zk_hat[(i*nd):((i+1)*nd)]
                xk[j,(i*nd):((i+1)*nd)] = xk[j,(i*nd):((i+1)*nd)] + mat_prod.dot(zk_diff)
            
    return xk

def max_likelihood(xk, fhat_model):
    t = xk.shape[0]
    
    M_new = np.zeros((d,d))
    for k in range(1, t-1):
        for i in range(m):
            fhat = fhat_model(xk[k,(i*nd):((i+1)*nd)].reshape(1,-1),k)
            xhat = xk[k+1,(i*nd):((i+1)*nd)]
            for j in range(n):
                fhatj = fhat[(j*d):((j+1)*d)]
                xhatj = xhat[(j*d):((j+1)*d)]
                sub = xhatj-fhatj
                outer = np.outer(sub, sub)
                M_new += outer
    M_new /= t*m*n
    return M_new


# this is the initial guess for the entropy matrix. can be pretty arbitrary
M = np.ones((d,d))
# this is the guess for the true error in the observations. should be small here.
Q = M * true_error**2

N_ITER = 5 # number of iterations of algorithm1_enks and max_likelihood 
Mhat = M
zk = observation(xt[:,:])
xm_hat = 0
xm_hat_prev = 0
for k in range(N_ITER):
    xm_hat = enks(zk, Mhat, Q, observation, lambda x,y: f_model(x,y));
    Mhat = max_likelihood(xm_hat, lambda x,y: f_model(x,y)) 
    print('current det(M)', np.linalg.det(Mhat))
    print('error change ', np.linalg.norm(xm_hat - xm_hat_prev)) 
    xm_hat_prev = xm_hat

In [ ]:
def entropy(M):
    return 1/2 * n * np.log((2*np.pi*np.exp(1))**d * np.linalg.det(M))
print('entropy(M estimated) ', entropy(Mhat)) 